In [76]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/hsuyeemon/SML_Project/main/data/life%20expectancy.csv')

Data Preprocessing

In [77]:
# we will use only the rows where the target Life Expectancy value present.
df = df[~df['Life Expectancy World Bank'].isna()]
print("Number of rows after dropping : ", df.shape[0])

Number of rows after dropping :  3118


In [78]:
percentage_missing = (df[['Health Expenditure %','Education Expenditure %']].isnull().sum() / len(df)) * 100
print("The missing value percent\n",percentage_missing)

The missing value percent
 Health Expenditure %        4.554201
Education Expenditure %    31.205901
dtype: float64


In [79]:
# TODO : // get a subset of data for project  

# Reference
# We will use the data from "East Asia & Pacific,South Asia and Europe & Central Asia" region since 2010.
#df = df[df['Year']>=2010]

# regions_filter = ['','']
# df = df[df['Region'].isin(regions_filter)]
# data = data[(data['Region'] == 'East Asia & Pacific') | (data['Region'] == 'South Asia')| (data['Region'] == 'Europe & Central Asia')]

In [80]:
df_filtered = df[~df['Country Name'].isin(['Dominica', 'Palau'])]

In [85]:
# Group by 'Country' and count the number of unique years
country_edu_data_counts = df_filtered[['Country Name','Education Expenditure %']].groupby('Country Name')['Education Expenditure %'].nunique() 

# Filter out countries with less than 5 years of data
countries_edu_with_less_than_5_years = country_edu_data_counts[country_edu_data_counts < 5].index.tolist()

# Filter out countries to impute
countries_edu_with_null_value = df_filtered[df_filtered['Education Expenditure %'].isna() == True]['Country Name'].unique().tolist()

print(countries_edu_with_less_than_5_years)
print(countries_edu_with_null_value)

['Bosnia and Herzegovina', 'Dominica', 'Equatorial Guinea', 'Eritrea', 'France', 'Greenland', 'Grenada', 'Guam', 'Iraq', 'Libya', 'Montenegro', 'Nigeria', 'North Macedonia', 'Palau', 'Papua New Guinea', 'Puerto Rico', 'Somalia', 'Turkmenistan', 'United Arab Emirates', 'United States']
['Afghanistan', 'Angola', 'United Arab Emirates', 'Antigua and Barbuda', 'Australia', 'Belgium', 'Burkina Faso', 'Bahrain', 'Bosnia and Herzegovina', 'Belarus', 'Bermuda', 'Bolivia', 'Botswana', 'Central African Republic', 'Chile', 'China', 'Comoros', 'Germany', 'Algeria', 'Ecuador', 'France', 'Gabon', 'Guinea-Bissau', 'Equatorial Guinea', 'Grenada', 'Greenland', 'Guatemala', 'Guam', 'Honduras', 'Croatia', 'Haiti', 'India', 'Iraq', 'Jordan', 'Kazakhstan', 'Liberia', 'Libya', 'Sri Lanka', 'Morocco', 'Maldives', 'North Macedonia', 'Malta', 'Myanmar', 'Montenegro', 'Mongolia', 'Mozambique', 'Nigeria', 'Nicaragua', 'Pakistan', 'Papua New Guinea', 'Puerto Rico', 'Sudan', 'Solomon Islands', 'El Salvador', 'Soma

In [86]:
# Group by 'Country' and count the number of unique years
country_hea_data_counts = df[['Country Name','Health Expenditure %']].groupby('Country Name')['Health Expenditure %'].nunique() 

# Filter out countries with less than 5 years of data
countries_hea_with_less_than_5_years = country_hea_data_counts[country_edu_data_counts < 5].index.tolist()

# Filter out countries to impute
countries_hea_with_null_value = df[df['Health Expenditure %'].isna() == True]['Country Name'].unique().tolist()


print(countries_hea_with_less_than_5_years)
print(countries_hea_with_null_value)

['Bosnia and Herzegovina', 'Dominica', 'Equatorial Guinea', 'Eritrea', 'France', 'Greenland', 'Grenada', 'Guam', 'Iraq', 'Libya', 'Montenegro', 'Nigeria', 'North Macedonia', 'Palau', 'Papua New Guinea', 'Puerto Rico', 'Somalia', 'Turkmenistan', 'United Arab Emirates', 'United States']
['Afghanistan', 'Bermuda', 'Greenland', 'Guam', 'Iraq', 'Montenegro', 'Puerto Rico', 'Somalia', 'South Sudan', 'Zimbabwe', 'Libya', 'Albania']


In [83]:
# handle missing data

df_filtered = df_filtered[~df_filtered['Country Name'].isin(countries_edu_with_less_than_5_years)]
print("data points after filtered" , df_filtered.shape[0])

df_filtered = df_filtered[~df_filtered['Country Name'].isin(countries_hea_with_less_than_5_years)]
print("data points after filtered" , df_filtered.shape[0])

data points after filtered 2774
data points after filtered 2774


In [84]:
# imputation

# Filter the DataFrame to include only the specified countries
df_subset = df_filtered[df_filtered['Country Name'].isin(countries_edu_with_null_value)]

# Calculate mean for each country group and impute missing values
#print(df_subset.groupby('Country Name')['Education Expenditure %'].transform(lambda x: x.fillna(x.mean())))
df_subset['Education Expenditure %'] = df_subset.groupby('Country Name')['Education Expenditure %'].transform(lambda x: x.fillna(x.mean()))

# Merge the subset back into the original DataFrame
df_imputed = pd.concat([df_filtered[~df_filtered['Country Name'].isin(countries_edu_with_null_value)], df_subset])

percentage_missing = (df_imputed[['Education Expenditure %']].isnull().sum() / len(df_imputed)) * 100
print("The missing value percent\n",percentage_missing)



# Filter the DataFrame to include only the specified countries
df_subset = df_imputed[df_imputed['Country Name'].isin(countries_hea_with_null_value)]

# Calculate mean for each country group and impute missing values
#print(df_subset.groupby('Country Name')['Education Expenditure %'].transform(lambda x: x.fillna(x.mean())))
df_subset['Health Expenditure %'] = df_subset.groupby('Country Name')['Health Expenditure %'].transform(lambda x: x.fillna(x.mean()))

# Merge the subset back into the original DataFrame
df_cleaned = pd.concat([df_imputed[~df_imputed['Country Name'].isin(countries_hea_with_null_value)], df_subset])

percentage_missing = (df_imputed[['Health Expenditure %']].isnull().sum() / len(df_imputed)) * 100
print("The missing value percent\n",percentage_missing)

The missing value percent
 Education Expenditure %    0.0
dtype: float64
The missing value percent
 Health Expenditure %    1.658255
dtype: float64


C:\Users\hsuye\AppData\Local\Temp\ipykernel_16324\3385512279.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['Education Expenditure %'] = df_subset.groupby('Country Name')['Education Expenditure %'].transform(lambda x: x.fillna(x.mean()))
C:\Users\hsuye\AppData\Local\Temp\ipykernel_16324\3385512279.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['Health Expenditure %'] = df_subset.groupby('Country Name')['Health Expenditure %'].transform(lambda x: x.fillna(x.mean()))


In [9]:
# Define the subset of countries we want to impute
countries_to_impute = [
    'Afghanistan','Albania','Angola','Australia','Austria','Bahrain',
    'Bangladesh','Barbados','Belarus','Belgium','Belize','Bhutan','Brazil',
    'Bulgaria','Burkina Faso','Burundi','Cambodia','Cameroon','Central African Republic',
    'Chad','Chile','China','Costa Rica','Croatia','Cuba','Cyprus',
    'Denmark','Djibouti','Dominican Republic','Ecuador','El Salvador',
    'Estonia','Eswatini','Ethiopia','Fiji','Finland','Gabon','Germany',
    'Ghana','Guatemala','Guinea','Guinea-Bissau','Guyana','Hungary',
    'Iceland','India','Indonesia','Ireland','Israel','Italy',
    'Jamaica','Japan','Jordan','Kazakhstan','Kenya','Kiribati',
    'Kuwait','Latvia','Lebanon','Lesotho','Lithuania',
    'Malawi','Malaysia','Maldives','Mali','Malta','Mauritania',
    'Mexico','Mongolia','Mozambique','Myanmar','Namibia',
    'Netherlands','New Zealand','Niger','Norway','Oman',
    'Pakistan','Panama','Paraguay','Philippines','Poland',
    'Portugal','Qatar','Romania','Rwanda','Samoa','Sao Tome and Principe',
    'Saudi Arabia','Senegal','Serbia','Seychelles','Sierra Leone',
    'Slovenia','Solomon Islands','Spain','Sri Lanka','Sweden',
    'Switzerland','Tajikistan','Tanzania','Trinidad and Tobago',
    'Tunisia','Uganda','United Kingdom','Uruguay','Vietnam','Zambia',
]


# Filter the DataFrame to include only the specified countries
df_subset = df_filtered[df_filtered['Country Name'].isin(countries_to_impute)]


# Calculate mean for each country group and impute missing values
#print(df_subset.groupby('Country Name')['Education Expenditure %'].transform(lambda x: x.fillna(x.mean())))
df_subset['Education Expenditure %'] = df_subset.groupby('Country Name')['Education Expenditure %'].transform(lambda x: x.fillna(x.mean()))

# Merge the subset back into the original DataFrame
df_imputed = pd.concat([df_filtered[~df_filtered['Country Name'].isin(countries_to_impute)], df_subset])


heath_imp = ['Afghanistan',
'Albania',
'Iraq',
'Zimbabwe',
]

# Filter the DataFrame to include only the specified countries
df_subset = df_imputed [df_imputed['Country Name'].isin(heath_imp)]


# Calculate mean for each country group and impute missing values
df_subset['Health Expenditure %'] = df_subset.groupby('Country Name')['Health Expenditure %'].transform(lambda x: x.fillna(x.mean()))


# Merge the subset back into the original DataFrame
df_imputed = pd.concat([df_imputed[~df_imputed['Country Name'].isin(heath_imp)], df_subset])


C:\Users\hsuye\AppData\Local\Temp\ipykernel_16324\2536136589.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['Education Expenditure %'] = df_subset.groupby('Country Name')['Education Expenditure %'].transform(lambda x: x.fillna(x.mean()))
C:\Users\hsuye\AppData\Local\Temp\ipykernel_16324\2536136589.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['Health Expenditure %'] = df_subset.groupby('Country Name')['Health Expenditure %'].transform(lambda x: x.fillna(x.mean()))


In [67]:
# get the columns of interest
df_cleaned = df_imputed[['Health Expenditure %','Education Expenditure %','Life Expectancy World Bank']]
print("The size of data is",df_cleaned.shape[0])
percentage_missing = (df_cleaned.isnull().sum() / len(df_cleaned)) * 100
print("The missing value percent\n",percentage_missing) 

The size of data is 2736
The missing value percent
 Health Expenditure %           0.000000
Education Expenditure %       23.099415
Life Expectancy World Bank     0.000000
dtype: float64


In [68]:
# remove outliers
df_cleaned = df_cleaned[(df_cleaned['Education Expenditure %'] <=8) & (df_cleaned['Health Expenditure %'] <= 12)]

In [69]:
X = df_cleaned[['Health Expenditure %','Education Expenditure %']] #input
y = df_cleaned['Life Expectancy World Bank'] #output

Splitting

In [70]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 42)

Training

In [71]:
from sklearn.preprocessing import StandardScaler,PolynomialFeatures
from sklearn.linear_model import Ridge
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

steps = [ ("scale", StandardScaler()),  #standardizing the data to range ( mean = 0 ,variance = 1)
          ("polytransform", PolynomialFeatures(degree = 3)), # transforming the polinomial to linear
          ("regressor", Ridge(1)) ] # Ridge Model

pipeline = Pipeline(steps)

model = pipeline.fit(X_train, y_train)  # goal :  to find the parameters , minimize the error between the error and actual data , doing optimization


Evaluation

In [72]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Evaluating on the training data
y_pred = model.predict(X_train)
mae = mean_absolute_error(y_train, y_pred)
mse = mean_squared_error(y_train, y_pred)
r2 = r2_score(y_train,y_pred)
print('training')
print('mae:', round(mae), 'mse:', round(mse), 'r2 score:', round(r2, 2))


# Evaluating on the testing data
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test,y_pred)
print('testing')
print('mae:', round(mae), 'mse:', round(mse), 'r2 score:', round(r2, 2))


training
mae: 6 mse: 61 r2 score: 0.26
testing
mae: 6 mse: 63 r2 score: 0.23
